*Analytical Information Systems*

# Tutorial 7 - Unsupervised Learning

Matthias Griebel<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

SS 2019

<h1>Agenda<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cluster-Analysis" data-toc-modified-id="Cluster-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cluster Analysis</a></span><ul class="toc-item"><li><span><a href="#Partitional-clustering:-k-Means-Algorithm" data-toc-modified-id="Partitional-clustering:-k-Means-Algorithm-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Partitional clustering: k-Means Algorithm</a></span><ul class="toc-item"><li><span><a href="#K-Means-Clustering-in-R" data-toc-modified-id="K-Means-Clustering-in-R-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>K-Means Clustering in R</a></span></li><li><span><a href="#Evaluating-K-means-Clusters" data-toc-modified-id="Evaluating-K-means-Clusters-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Evaluating K-means Clusters</a></span></li></ul></li><li><span><a href="#Hierarchical-Clustering" data-toc-modified-id="Hierarchical-Clustering-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Hierarchical Clustering</a></span><ul class="toc-item"><li><span><a href="#Hierarchical-Clustering-in-R" data-toc-modified-id="Hierarchical-Clustering-in-R-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Hierarchical Clustering in R</a></span></li></ul></li></ul></li><li><span><a href="#Principal-Components-Analysis" data-toc-modified-id="Principal-Components-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Principal Components Analysis</a></span><ul class="toc-item"><li><span><a href="#PCA-in-R" data-toc-modified-id="PCA-in-R-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>PCA in R</a></span></li><li><span><a href="#Cluster-visualization-using-PCA" data-toc-modified-id="Cluster-visualization-using-PCA-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Cluster visualization using PCA</a></span></li></ul></li><li><span><a href="#Exam-Questions" data-toc-modified-id="Exam-Questions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exam Questions</a></span><ul class="toc-item"><li><span><a href="#Exam-AIS-SS-2018" data-toc-modified-id="Exam-AIS-SS-2018-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Exam AIS SS 2018</a></span></li></ul></li></ul></div>

__Supervised learning__
- Predict Y using X data
 
__Unsupervised learning__
- Only X features observed
- Goal is to discover interesting things about the measurements
- __Cluster Analysis__
    - Can we discover subgroups among the variables or among the observations?
- __Dimensionality Reduction__
    - Can we improve information density of the data – are there redundant variables?


## Cluster Analysis

__What is Cluster Analysis/Clustering?__

- __Cluster__: a collection of data objects that are
    - similar to one another within the same cluster
    - dissimilar to the objects in other clusters


- __Cluster analysis__
    - grouping a set of data objects into clusters

<img src='images/07/cluster_analysis.png' >

__Types of Clusterings__

- __Partitional Clustering__
    - a division data objects into non-overlapping subsets (clusters) such that each data object is in exactly one subset
- __Hierarchical clustering__
    - a set of nested clusters organized as a hierarchical tree

Load packages for today

In [ ]:
library(tidyverse)
library(tidymodels)
library(ggrepel)
library(ggfortify)
library(ggdendro)

__The protein dataset__

The data set contains data about 25 European countries and their protein intakes (in percent) from nine major food sources (p = 9).

In [ ]:
url = 'http://www.biz.uiowa.edu/faculty/jledolter/DataMining/protein.csv'
food <- read_csv(url)

In [ ]:
food %>% head()

Can we discover subgroups among the countries regarding their major food sources?

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
food %>%
    ggplot(aes(RedMeat, WhiteMeat)) + 
    geom_point() + 
    geom_label_repel(aes(label=Country), size = 2) +
    theme_bw()

### Partitional clustering: k-Means Algorithm

- Each cluster is associated with a centroid (center point)
- Each point is assigned to the cluster with the closest centroid
    - ‘Closeness’ is measured by Euclidean distance, cosine similarity, correlation, etc.
- Number of clusters, K, must be specified
    - Initial centroids are often chosen randomly - clusters may vary from one run to another


__Objective:__ Minimize the within-cluster sum of squares (i.e. variance)
$$\underset{\mathbf{C}}{arg\,min} = \sum_{i=1}^{K}\sum_{x\in C_i}dist^2(m_i,x)$$

The basic k-means algorithm does not guarantee to find the optimum

<img src='images/07/k-means.png' style="width:50%" >

#### K-Means Clustering in R

`kmeans()` performs a K-Means clustering on a data matrix.

Important parameters:

- `centers`:  Either the number of clusters, say 𝑘, or a set of initial (distinct) cluster centroids
- `iter.max`: The maximum number of iterations allowed.
- `nstart`:  How many random initial centroid configurations should be chosen? Reports on the best one.

In [ ]:
?kmeans

First, we start clustering on just Red and White meat with k=3 clusters.

In [ ]:
food_meat <- food %>% select(RedMeat, WhiteMeat)
km_meat <- kmeans(food_meat, centers = 3, nstart = 10)
km_meat

Let's visualize the results
- `augment` adds the clustering result to the original dataset

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
km_meat %>%
    augment(food) %>%
    ggplot(aes(x=RedMeat, y=WhiteMeat, color=.cluster)) +
    geom_point() +
    geom_label_repel(aes(label=Country), size = 3) +
    theme_bw()

#### Evaluating K-means Clusters

- `between_SS`: The between-cluster sum of squares
    -  measures heterogeneity between the clusters

- `within_SS`: Vector of within-cluster sum of squares, one component per cluster.
    -  measures homogeneity within the clusters  

- `total_SS`= `between_SS` + $\sum$ `within_SS`

- Ratio `between_SS` / `total_SS`
    - Properties of internal cohesion and external separation
    - Ideally ratio should approach 1 (relatively small $\sum$ `within_SS`)

- `glance` extracts a single-row summary

In [ ]:
glance(km_meat)

- `tidy` summarizes on a per-cluster level

In [ ]:
tidy(km_meat)

Let's explore the effect of different choices of k, from 1 to 9, on this clustering.
1. Cluster the data 9 times, each using a different value of k
2. Create columns containing the tidied, glanced and augmented data

In [ ]:
k_clusts <- tibble(k = 1:9) %>%
  mutate(
    kclust = map(k, ~kmeans(food_meat, .x)),
    tidied = map(kclust, tidy),
    glanced = map(kclust, glance),
    augmented = map(kclust, augment, food)
  )

In [ ]:
clusters <- k_clusts %>%
  unnest(tidied)

assignments <- k_clusts %>% 
  unnest(augmented)

clusterings <- k_clusts %>%
    unnest(glanced, .drop=TRUE)

Plot the different clusters

In [ ]:
assignments %>%
    ggplot(aes(RedMeat, WhiteMeat)) +
    geom_point(aes(color = .cluster)) + 
    facet_wrap(~ k) +
    geom_point(data = clusters, aes(x1, x2), size = 5, shape = "x") +
    theme_bw()

__Elbow method to determine the optimal number of clusters for k-means clustering__
- Idea: One should choose a number of clusters so that adding another cluster doesn't give much better modeling of the data
- There are many more metrics that try to define the optimal number of cluster (see `NbClust` package)

In [ ]:
clusterings %>%
    ggplot(aes(x=k, y=tot.withinss)) + 
    geom_line() +
    scale_x_continuous(breaks = c(1:9)) +
    geom_vline(xintercept = 3, linetype = "dashed")

Now, we want to include all variables

In [ ]:
km_protein <- food %>%
    select(-Country) %>%
    kmeans(centers=3, nstart=10)

Let's visualize the results

In [ ]:
km_protein %>% 
    augment(food) %>%
    ggplot(aes(x=RedMeat, y=WhiteMeat, color=.cluster)) +
    geom_point() +
    geom_label_repel(aes(label=Country), size= 2) +
    theme_bw()

Is there a better way to visualize the results?

### Hierarchical Clustering

Produce a nested sequence of clusters, a tree, also called Dendrogram

- __Agglomerative (bottom up) clustering__
    - builds the dendrogram (tree) from the bottom level
    - merges the most similar (or nearest) pair of clusters 
    - stops when all the data points are merged into a single cluster
    
- __Divisive (top down) clustering__
    - starts with all data points in one cluster, the root
    - splits the root into a set of child clusters
    - stops when only singleton clusters of individual data points remain

#### Hierarchical Clustering in R
- `hclust()` performs an agglomerative hierarchical cluster analysis on a set of dissimilarities 
- `dist()` computes and returns the distance matrix by using the specified distance measure (i.e., euclidean diestance)
- `ggdendrogram()` creates a dendrogram plot using `ggplot`

In [ ]:
food %>%
    column_to_rownames("Country") %>%
    dist(method = "euclidean") %>%
    hclust(method = "ward.D") -> h_protein
h_protein

Plot the dendrogram

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
h_protein  %>% 
    ggdendrogram(rotate = T)

__Get clusters__

`cutree` cuts a tree, e.g., as resulting from `hclust`, into several groups either by specifying the desired number(s) of groups or the cut height(s).

In [ ]:
h_results <- cutree(h_protein, k=3)
h_results

Let's again visualize the results. Is there a better way?

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
food %>%
    mutate(h_clusters = as.factor(h_results)) %>%    
    ggplot(aes(x=RedMeat, y=WhiteMeat, color=h_clusters)) +
    geom_point() +
    geom_label_repel(aes(label=Country), size= 2) +
    theme_bw()

## Principal Components Analysis

__Principal Components Analysis__

__PCA__ produces a low-dimensional representation of a dataset
- sequence of linear combinations of the variables that have maximal variance, and are mutually uncorrelated

We use __PCA__ for 
 - data visualization
 - data pre-processing before supervised techniques are applied


__PCA vs Clustering__
- PCA looks for a low-dimensional representation of the observations that explains a good fraction of the variance
- Clustering looks for homogeneous subgroups among the observations

###  PCA in R

`prcomp` performs a principal components analysis on the given data matrix

In [ ]:
food %>%
    select(-Country) %>%
    prcomp(center = T, scale. = T) -> pca_protein
pca_protein

__Scree-plot__

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
screeplot(pca_protein)

Rather use `ggplot`

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
tidy(pca_protein, "pcs") %>%
    ggplot(aes(x=PC, y=std.dev^2)) + 
    geom_line() +
    geom_point() + 
    scale_x_continuous(breaks = c(1:9)) +
    theme_bw()

### Cluster visualization using PCA

Let's visualize the __K-Means__ results. Why is scaling the data so important?

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
pca_protein %>%
        autoplot(x = 1,
                 y = 2,
                 data = data.frame(Cluster = as.factor(km_protein$cluster)),
                 colour = "Cluster",
                 loadings = TRUE,
                 loadings.label = TRUE,
                 loadings.colour = 'black',
                 loadings.label.colour =  'black',
                 loadings.label.repel = T,
                 frame = TRUE) + 
    geom_label_repel(aes(label=food$Country), size = 3, alpha=0.5) +
    theme_bw()

Let's visualize the results of __hierarchical clustering__

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
pca_protein %>%
        autoplot(x = 1,
                 y = 2,
                 data = data.frame(Cluster = as.factor(h_results)),
                 colour = "Cluster",
                 loadings = TRUE,
                 loadings.label = TRUE,
                 loadings.colour = 'black',
                 loadings.label.colour =  'black',
                 loadings.label.repel = T,
                 frame = TRUE) + 
    geom_label_repel(aes(label=food$Country), size = 3, alpha=0.5) +
    theme_bw()

## Exam Questions

### Exam AIS SS 2018

__Question 4: Unsupervised Learning__

(c) __PCA__ You performed a principal component analysis (PCA) on a dataset of 200 military ships of the Second World War. The first four components are displayed in the following table

\begin{array}{l|rrrrl}
\hline
                        & Components     \\ \hline
                        & 1          & 2     & 3     & 4     & ... \\ \hline
Water \ displacement      & 0.95       & -0.09 & −0.12 & 0.22  & ... \\
Length                  & 0.90       & 0.30  & −0.06 & −0.20 & ... \\
Width                   & 0.97       & −0.12 & −0.03 & 0.03  & ... \\
Engine \ power            & 0.55       & 0.77  & −0.19 & −0.13 & ... \\
Speed                   & −0.52      & 0.79  & −0.15 & 0.22  & ... \\
Radius \ of \ action        & 0.39       & 0.31  & 0.86  & 0.03  & ... \\
Crew \ size               & 0.95       & 0.06  & −0.05 & 0.10  & ... \\ \hline
\textbf{% of variance} & 64.88      & 19.22 & 10.43 & 2.22  & ... \\ \hline
\end{array}

i. (1 point) How many principal components are there in total? Briefly explain the number.

>

(ii.) (2 points) A PCA is particularly helpful if it is possible to attribute a concept or interpretation to a component. Provide a meaningful explanation of the concepts embedded in principal component 1 and 2 of the ship data.

> 

iii. (3 points) How would you proceed to use the PCA for dimensionality reduction purposes?

> 